In [157]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load Data

In [122]:
prices = pd.read_csv("data/prices_panel.csv", parse_dates = ["Date"])
funds = pd.read_csv("data/fundamentals_income.csv", parse_dates = ["Report Date", "Publish Date", "Restated Date"])

# change column names so it easier to type / access them
prices.columns = [c.strip().lower().replace(' ', '_').replace('.', '') for c in prices.columns]
funds.columns = [c.strip().lower().replace(' ', '_').replace('.', '').replace('(', '').replace(')', '').replace(',', '') for c in funds.columns]

In [123]:
# Drop duplicates

prices = prices.sort_values(["ticker", "date"]).drop_duplicates()
funds = funds.sort_values(["ticker", "report_date"]).drop_duplicates()

In [124]:
prices.head()
funds.head()

,index,ticker,simfinid,currency,fiscal_year,fiscal_period,report_date,publish_date,restated_date,shares_basic,...,non-operating_income_loss,interest_expense_net,pretax_income_loss_adj,abnormal_gains_losses,pretax_income_loss,income_tax_expense_benefit_net,income_loss_from_continuing_operations,net_extraordinary_gains_losses,net_income,net_income_common
0,131,AAPL,111052,USD,2020,Q1,2019-12-31,2020-01-29,2021-01-28,1.766016e+10,...,349000000.0,NaN,2.591800e+10,NaN,25918000000,-3.682000e+09,22236000000,NaN,22236000000,22236000000
1,132,AAPL,111052,USD,2020,Q2,2020-03-31,2020-05-01,2021-04-29,1.744040e+10,...,282000000.0,NaN,1.313500e+10,NaN,13135000000,-1.886000e+09,11249000000,NaN,11249000000,11249000000
2,133,AAPL,111052,USD,2020,Q3,2020-06-30,2020-07-31,2021-07-28,1.725029e+10,...,46000000.0,NaN,1.313700e+10,NaN,13137000000,-1.884000e+09,11253000000,NaN,11253000000,11253000000
3,134,AAPL,111052,USD,2020,Q4,2020-09-30,2020-10-30,2021-07-28,1.705762e+10,...,126000000.0,NaN,1.490100e+10,NaN,14901000000,-2.228000e+09,12673000000,NaN,12673000000,12673000000
4,135,AAPL,111052,USD,2021,Q1,2020-12-31,2021-01-28,2022-01-28,1.693512e+10,...,45000000.0,NaN,3.357900e+10,NaN,33579000000,-4.824000e+09,28755000000,NaN,28755000000,28755000000


# Feature Selection

In [125]:
prices["ret"] = prices.groupby("ticker")['adj_close'].pct_change()

In [126]:
# Build fundamental features

funds['eps'] = funds["net_income"] / funds['shares_diluted']
funds['profit_margin'] = funds['net_income'] / funds['revenue']
funds['revenue_growth'] = funds.groupby('ticker')['revenue'].pct_change()

In [127]:
prices = prices.sort_values(["date", "ticker"]).reset_index(drop=True)
funds = funds.sort_values(["report_date", "ticker"]).reset_index(drop=True)

df = pd.merge_asof(
    prices,
    funds,
    by="ticker",
    left_on="date",
    right_on="report_date",
    direction="backward",
)

df.head()

,index_x,ticker,simfinid_x,date,open,high,low,close,adj_close,volume,...,abnormal_gains_losses,pretax_income_loss,income_tax_expense_benefit_net,income_loss_from_continuing_operations,net_extraordinary_gains_losses,net_income,net_income_common,eps,profit_margin,revenue_growth
0,16033,AAPL,111052,2019-12-23,70.13,71.06,70.09,71.00,68.53,98711532,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40152,ABT,63877,2019-12-23,86.31,87.46,86.28,87.35,78.56,4067769,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,95854,ADBE,14099,2019-12-23,328.83,329.88,327.26,328.95,328.95,2210706,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,296720,AMGN,65735,2019-12-23,243.98,244.00,241.59,243.03,203.47,1686220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,333254,AMZN,62747,2019-12-23,89.41,89.65,89.23,89.65,89.65,42749860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
g = df.groupby("ticker")

df['mean_20'] = g['ret'].transform(lambda x: x.rolling(20, 10).mean())
df['mean_60'] = g['ret'].transform(lambda x: x.rolling(60, 20).mean())

df['vol_20'] = g['ret'].transform(lambda x: x.rolling(20, 10).std())
df['vol_60'] = g['ret'].transform(lambda x: x.rolling(60, 20).std())

<h3 style="color:red">Todo: generate more features</h3>

Select features

In [136]:
df.columns

Index(['index_x', 'ticker', 'simfinid_x', 'date', 'open', 'high', 'low',
       'close', 'adj_close', 'volume', 'dividend', 'shares_outstanding', 'ret',
       'index_y', 'simfinid_y', 'currency', 'fiscal_year', 'fiscal_period',
       'report_date', 'publish_date', 'restated_date', 'shares_basic',
       'shares_diluted', 'revenue', 'cost_of_revenue', 'gross_profit',
       'operating_expenses', 'selling_general_&_administrative',
       'research_&_development', 'depreciation_&_amortization',
       'operating_income_loss', 'non-operating_income_loss',
       'interest_expense_net', 'pretax_income_loss_adj',
       'abnormal_gains_losses', 'pretax_income_loss',
       'income_tax_expense_benefit_net',
       'income_loss_from_continuing_operations',
       'net_extraordinary_gains_losses', 'net_income', 'net_income_common',
       'eps', 'profit_margin', 'revenue_growth', 'mean_20', 'mean_60',
       'vol_20', 'vol_60', 'cumret_1', 'y_1', 'cumret_20', 'y_20', 'cumret_60',
       'y_6

In [137]:
raw = ['ret']
eng = ['mean_20', 'mean_60', 'vol_20', 'vol_60']
fund = ['eps', 'profit_margin', 'revenue_growth']
fund_eng = fund + eng

# Feature Selection

In [129]:
rt = df.pivot(index='date', columns='ticker', values='ret')
rt

ticker,AAPL,ABT,ADBE,AMGN,AMZN,AXP,BAC,BMY,CMCSA,COST,...,RTX,SBUX,T,TSLA,TXN,UNH,V,VZ,WMT,XOM
date,,,,,,,,,,,,,,,,,,,,,
2019-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-24,0.000876,-0.000764,0.002098,-0.002851,-0.002119,0.002001,0.001639,-0.000793,0.009595,0.003130,...,-0.001544,0.003367,-0.002459,0.014311,-0.000185,-0.001856,0.002621,-0.002067,0.004106,-0.003761
2019-12-26,0.019974,0.000000,0.004732,-0.001774,0.044489,0.005382,0.008508,0.012691,0.010032,0.005126,...,0.007577,-0.005033,0.004931,0.013404,-0.000369,0.003756,0.008456,0.000230,0.000000,0.001510
2019-12-27,-0.000429,0.001274,-0.001238,-0.001531,0.000535,-0.001813,-0.004867,0.002937,0.002875,-0.005506,...,-0.002916,0.000519,0.001963,-0.001392,0.000739,0.001112,0.001214,0.003911,0.000818,-0.003392
2019-12-30,0.005863,-0.006870,-0.007407,-0.005242,-0.012301,-0.007093,-0.005543,-0.006443,0.001564,0.003530,...,-0.002617,-0.007778,-0.004897,-0.036250,-0.007105,-0.007180,-0.008264,-0.005270,-0.001634,-0.005862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-19,0.001145,-0.001911,0.000200,0.004266,0.014427,0.000035,-0.006572,0.025139,-0.015927,0.011577,...,-0.003655,-0.020354,-0.010909,0.021432,-0.029504,-0.021454,-0.000968,-0.007589,0.029903,-0.013961
2024-11-20,0.003168,-0.010267,-0.000220,0.028309,-0.008455,0.007541,-0.007718,-0.005950,0.015939,-0.002226,...,0.001621,-0.000940,0.004136,-0.011474,-0.014328,0.040732,-0.014306,0.006882,0.006763,0.014246
2024-11-21,-0.002105,0.011429,0.009890,0.007052,-0.022181,0.018412,0.008889,0.005986,0.011827,0.029698,...,0.011582,0.018288,0.006407,-0.006988,0.000052,-0.005003,0.008158,0.006582,0.013899,0.013442


<h3 style="color:red">Todo: save clean data for easy access</h3>

# Binary Labels

In [154]:
# NOTE: in here we are deleting observation which might be a problem

labels = [1, 20, 60]

g = df.groupby('ticker')

for h in labels:
    df[f'cumret_{h}'] = g['ret'].transform(lambda x: (1 + x).rolling(h).apply(lambda r: np.prod(r) - 1).shift(-h + 1))
    
df = df.dropna(subset=[f'cumret_{h}' for h in labels])

for h in labels:
    df[f'y_{h}'] = (df[f'cumret_{h}'] > 0).astype(int)

df

/var/folders/bc/hnzwjdn546lcc572zg36k_vc0000gn/T/ipykernel_14153/3986904653.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'cumret_{h}'] = g['ret'].transform(lambda x: (1 + x).rolling(h).apply(lambda r: np.prod(r) - 1).shift(-h + 1))
/var/folders/bc/hnzwjdn546lcc572zg36k_vc0000gn/T/ipykernel_14153/3986904653.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'y_{h}'] = (df[f'cumret_{h}'] > 0).astype(int)
/var/folders/bc/hnzwjdn546lcc572zg36k_vc0000gn/T/ipykernel_14153/3986904653.py:13: Settin

,index_x,ticker,simfinid_x,date,open,high,low,close,adj_close,volume,...,mean_20,mean_60,vol_20,vol_60,cumret_1,y_1,cumret_20,y_20,cumret_60,y_60
49,16034,AAPL,111052,2019-12-24,71.17,71.22,70.73,71.07,68.59,48478856,...,NaN,NaN,NaN,NaN,0.000876,1,0.124033,1,-0.190865,0
50,40153,ABT,63877,2019-12-24,87.37,87.48,86.98,87.28,78.50,1067679,...,NaN,NaN,NaN,NaN,-0.000764,0,0.043024,1,-0.218304,0
51,95855,ADBE,14099,2019-12-24,329.00,331.54,328.68,329.64,329.64,1066406,...,NaN,NaN,NaN,NaN,0.002098,1,0.069342,1,-0.102174,0
52,296721,AMGN,65735,2019-12-24,242.82,243.10,241.72,242.33,202.89,612809,...,NaN,NaN,NaN,NaN,-0.002851,0,-0.032830,0,-0.219836,0
53,333255,AMZN,62747,2019-12-24,89.69,89.78,89.38,89.46,89.46,17626740,...,NaN,NaN,NaN,NaN,-0.002119,0,0.051088,1,0.029559,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54973,5695189,UNH,367714,2024-06-07,502.04,502.86,490.42,490.69,476.46,3416349,...,-0.001532,0.000226,0.014733,0.016508,-0.022365,0,-0.020868,0,0.197903,1
54974,5755878,V,107326,2024-06-07,277.71,280.33,276.86,278.67,275.63,4515795,...,0.000143,-0.000320,0.007552,0.007965,0.005875,1,-0.038391,0,0.007408,1
54975,5926759,VZ,101219,2024-06-07,41.16,41.29,40.81,40.94,37.11,10410163,...,0.001483,0.000618,0.011354,0.011871,-0.009343,0,0.004805,1,0.055526,1
54976,6024165,WMT,239962,2024-06-07,67.40,67.56,65.64,65.88,65.06,20213211,...,0.004457,0.001388,0.017491,0.011910,-0.018851,0,0.037702,1,0.152466,1


# Modeling

In [139]:
dates = df['date'].sort_values().unique()
N = len(dates)

train_dt = dates[int(0.70 * N)]
val_dt = dates[int(0.85 * N)]

train = df[df['date'] <= train_dt]
val = df[(df['date'] > train_dt) & (df['date'] <= val_dt)]
test = df[df['date'] > val_dt]

## Standarization

In [155]:
X_train = train[fund_eng]
X_val = val[fund_eng]
X_test = test[fund_eng]

scaler = StandardScaler()
scaler.fit(X_train)

,copy,True
,with_mean,True
,with_std,True


In [156]:
X_train = scaler.transform(X_train)
X_val   = scaler.transform(X_val)
X_test  = scaler.transform(X_test)

## Training

<h3 style="color:red">Note: apply basic logistic regression as starting point - ideally do LSTM later</h3>

In [158]:
target = "y_20"

y_train = train[target]
y_val   = val[target]
y_test  = test[target]

model = LogisticRegression(max_iter = 1000)
model.fit(X_train, y_train)
acc_val = accuracy_score(val[target], model.predict(X_val))
acc_test = accuracy_score(test[target], model.predict(X_test))

print(acc_val, acc_test)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values